#Simulating Users

### Possible Topics
* A/B Testing (notion vs truth)
    * Diagram of simple vs Bing
* Problem statement (it's one thing to to execute a plan, it's another to achieve robust/trustworthy exp)
* Design vs Execution vs Scorecard
    * Design can add complexity
    * Execution is not analysis
    * Scorecard is a "view" not the truth
* Execution to Scorecard phase:
    * Empirically has had problems
    * Many are just incorrect execution
    * Some are incorrect aggregation
    * Most were fixable with unit testing
* Scaling and testing:
    * Don't need actual experiments...we can simulate
* Simulation
    * Humans are suprisingly simple when it comes to log lines
    * Artifact of what we log to some extent
* Simple flow:
    1. Human visits
    2. Human has choice (often influenced by treatment...you hope)
    3. Human makes choice
    4. (Optional) Human repeats 2 and 3 additional times
    * Key point: Logs are *generated* by a process
* Abstract the process:
    1. Present a choice (probability distribution)
    2. Draw from that distribution
    3. Given the draw, present a second choice (another probability distribution, possibly different)
    4. Draw again
    5. Repeat
* 